#### Imports and functions

In [52]:
import pandas as pd
import tensorflow as tf
import gdown
import numpy as np
import matplotlib.pyplot as plt

import cv2 as cv

folder_url = "https://drive.google.com/drive/folders/19toF0-LXqLJwthD2ER6WNGkpQnPW6tYB?usp=share_link"
folder_id = "19toF0-LXqLJwthD2ER6WNGkpQnPW6tYB?usp=sharing"
gdown.download_folder(id=folder_id)

In [49]:
def normalize_test(input_image):
    input_image = tf.cast(input_image, tf.float32)
    input_image = (input_image / 127.5) - 1

    return input_image

def generate_test_images(model, test_input):
    prediction = model(test_input, training=True)
    return(prediction[0])

def submission_data(test_path):
    test_mask_data = pd.read_csv(test_path + '/masked_info.csv') #.drop(['Unnamed: 0'],axis=1)
    submission = []
    for i, img in enumerate(test_dataset):
        filename = file_paths[i].split('/')[-1]
        y1, x1, y2, x2 = test_mask_data[test_mask_data['filename']==filename].iloc[0,2:]
        tensor_array = generate_test_images(generator, img)
        im = np.array(tensor_array)
        b, g, r = cv.split(im)
        result = np.zeros((256, 256))
        normalized_b = cv.normalize(b,result, 0, 1, cv.NORM_MINMAX, dtype=cv.CV_32F)
        normalized_g = cv.normalize(g,result, 0, 1, cv.NORM_MINMAX, dtype=cv.CV_32F)
        normalized_r = cv.normalize(r,result, 0, 1, cv.NORM_MINMAX, dtype=cv.CV_32F)
        for i in range(y1 , y1+75):
            for j in range(x1, x1+75):
                temp1 = filename + '_' + 'box1' + '_' +  str(i) + '_' + str(j) + '_'
                submission.append((temp1 + '0', normalized_b[i,j]))
                submission.append((temp1 + '1', normalized_g[i,j]))
                submission.append((temp1 + '2', normalized_r[i,j]))
        for i in range(y2 , y2+75):
            for j in range(x2, x2+75):
                temp1 = filename + '_' + 'box2' + '_' + str(i) + '_' + str(j) + '_'
                submission.append((temp1 + '0', normalized_b[i,j]))
                submission.append((temp1 + '1', normalized_g[i,j]))
                submission.append((temp1 + '2', normalized_r[i,j]))
    df = pd.DataFrame(submission, columns=['filename_box_pixel','value'])
    return(df)

#### Inference

In [ ]:
## INSERT THE TEST PATH HERE

test_path = "-- test path --"

# please ensure there's a masked_info.csv file inside the test path

In [42]:
generator = tf.keras.models.load_model('./generator_model')
test_dataset = tf.keras.utils.image_dataset_from_directory(directory=test_path, 
                                                batch_size=1, image_size=[256, 256], labels="inferred", 
                                                label_mode=None, shuffle=False)
                                                
file_paths = test_dataset.file_paths
AUTOTUNE = tf.data.AUTOTUNE
test_dataset = test_dataset.map(normalize_test, num_parallel_calls=AUTOTUNE)

Found 200 files belonging to 1 classes.


#### Submission file generation

In [53]:
df = submission_data(test_path)
df.to_csv("submission_group24.csv",index=False)
df.head()